<div>
    <img src="img/AF-logotiposecundario-GiantSteps-positivo-2.png" width="250px">
</div>

In [188]:
import pandas as pd, urllib.request, json, numpy as np, warnings
from tkinter import Tk, ttk, StringVar, IntVar, messagebox
from tkcalendar import DateEntry
from datetime import datetime
from dateutil import parser

warnings.filterwarnings('ignore')
std_date_format = '%d/%m/%Y'

In [189]:
def GUI(
    START_DATE=datetime(1995, 1, 1),
    END_DATE=datetime.now(),
    PERIOD='DAY',
    CAPITAL=100,
    save_csv_i=False,
    save_xlsx_i=False,
):
    window = Tk()
    window.title('Giant Steps Capital Internship Technical Challenge')
    window.resizable(False, False)
    window.eval('tk::PlaceWindow . center')
    window.call('wm', 'attributes', '.', '-topmost', True)

    START_DATE_FINAL, END_DATE_FINAL, PERIOD_FINAL, CAPITAL_FINAL = (
        START_DATE,
        END_DATE,
        PERIOD,
        CAPITAL,
    )
    save_csv, save_xlsx = save_csv_i, save_xlsx_i

    START_DATE_ENTRY = DateEntry(
        justify='center', mindate=START_DATE, date_pattern='dd/mm/y'
    )
    START_DATE_ENTRY.set_date('01/01/1995')
    START_DATE_LABEL = ttk.Label(text='Insert the start date:', justify='center')

    END_DATE_ENTRY = DateEntry(
        justify='center', maxdate=END_DATE, date_pattern='dd/mm/y'
    )
    END_DATE_ENTRY.set_date(END_DATE.strftime(std_date_format))
    END_DATE_LABEL = ttk.Label(text='Insert the end date:', justify='center')

    PERIOD_VAR = StringVar(window)
    PERIOD_VAR.set(PERIOD)
    PERIOD_LABEL = ttk.Label(text='Select the frequency:', justify='center')
    period_list = ['DAY', 'MONTH', 'YEAR']
    PERIOD_ENTRY = ttk.OptionMenu(window, PERIOD_VAR, PERIOD, *period_list)

    CAPITAL_ENTRY = ttk.Entry(justify='center', exportselection=0, width=10)
    CAPITAL_ENTRY.insert(-1, CAPITAL)
    CAPITAL_LABEL = ttk.Label(
        text='Insert the initial capital amount:', justify='center'
    )
    
    save_csv_var = IntVar(value=0)
    save_xlsx_var = IntVar(value=0)

    save_csv_button = ttk.Checkbutton(
        window, text='Save .csv file', variable=save_csv_var, onvalue=1, offvalue=0
    )
    save_xlsx_button = ttk.Checkbutton(
        window, text='Save .xlsx file', variable=save_xlsx_var, onvalue=1, offvalue=0
    )

    def isClickedFunction():
        nonlocal START_DATE_FINAL, END_DATE_FINAL, PERIOD_FINAL, CAPITAL_FINAL, save_csv, save_xlsx

        START_DATE_FINAL = START_DATE_ENTRY.get()
        END_DATE_FINAL = END_DATE_ENTRY.get()
        PERIOD_FINAL = PERIOD_VAR.get()
        CAPITAL_FINAL = CAPITAL_ENTRY.get()
        save_csv = save_csv_var.get()
        save_xlsx = save_xlsx_var.get()

        window.destroy()

    BUTTON = ttk.Button(text='Search', command=isClickedFunction)

    PERIOD_LABEL.grid(row=1, column=1, pady=10)
    PERIOD_ENTRY.grid(row=2, column=1, padx=25)

    START_DATE_LABEL.grid(row=3, column=0, pady=10)
    START_DATE_ENTRY.grid(row=4, column=0, padx=25)

    END_DATE_LABEL.grid(row=3, column=2, pady=10)
    END_DATE_ENTRY.grid(row=4, column=2, padx=25)

    CAPITAL_LABEL.grid(row=3, column=1, pady=10)
    CAPITAL_ENTRY.grid(row=4, column=1, padx=25)

    BUTTON.grid(row=5, column=1, pady=15)

    save_csv_button.grid(row=5, column=0, pady=15)
    save_xlsx_button.grid(row=5, column=2, pady=15)

    window.mainloop()

    return (
        START_DATE_FINAL,
        END_DATE_FINAL,
        PERIOD_FINAL,
        CAPITAL_FINAL,
        save_csv,
        save_xlsx,
    )
    
def filename_GUI():
    window = Tk()
    window.title('Giant Steps Capital Internship Technical Challenge')
    window.resizable(False, False)
    window.eval('tk::PlaceWindow . center')
    window.call('wm', 'attributes', '.', '-topmost', True)
    
    FILENAME_FINAL = None

    FILENAME_LABEL = ttk.Label(text = 'Insert the desired filename for the export:', justify = 'center')
    FILENAME_ENTRY = ttk.Entry(justify = 'center')
    
    def isClickedFunction():
        nonlocal FILENAME_FINAL
        
        FILENAME_FINAL = FILENAME_ENTRY.get()
        
        window.destroy()
    
    BUTTON = ttk.Button(text = 'Submit', command = isClickedFunction)
    
    FILENAME_LABEL.grid(row=1, column=2, pady=10, padx=25)
    FILENAME_ENTRY.grid(row=2, column=2, padx=25)
    
    BUTTON.grid(row=4, column=2, pady=15)
    
    window.mainloop()
    
    return FILENAME_FINAL

In [190]:
(
    START_DATE_FINAL,
    END_DATE_FINAL,
    PERIOD_FINAL,
    CAPITAL_FINAL,
    save_csv,
    save_xlsx,
) = GUI()
CAPITAL_FINAL = float(CAPITAL_FINAL)

while True:
    if parser.parse(START_DATE_FINAL) > parser.parse(END_DATE_FINAL):
        root = Tk()
        root.withdraw()
        messagebox.showerror('Error', 'End date must not exceed start date')
        root.destroy()
        (
            START_DATE_FINAL,
            END_DATE_FINAL,
            PERIOD_FINAL,
            CAPITAL_FINAL,
            save_csv,
            save_xlsx,
        ) = GUI(
            PERIOD=PERIOD_FINAL,
            CAPITAL=CAPITAL_FINAL,
            save_csv_i=save_csv,
            save_xlsx_i=save_xlsx,
        )
    else:
        break
    
if save_xlsx or save_csv:
    filename = filename_GUI()

In [191]:
initial_day = START_DATE_FINAL.split('/')[0]
initial_month = START_DATE_FINAL.split('/')[1]
initial_year = START_DATE_FINAL.split('/')[2]

final_day = END_DATE_FINAL.split('/')[0]
final_month = END_DATE_FINAL.split('/')[1]
final_year = END_DATE_FINAL.split('/')[2]

In [192]:
selic_rates_base_url = r'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json&dataInicial=dd/mm/yy&dataFinal=dd/mm/yy'
selic_rates_url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json&dataInicial={initial_day}/{initial_month}/{initial_year}&dataFinal={final_day}/{final_month}/{final_year}'

with urllib.request.urlopen(selic_rates_url) as url:
    data = json.loads(url.read().decode())
    selic_rates = pd.DataFrame(data)

selic_rates['data'] = pd.to_datetime(selic_rates['data'], format=std_date_format)
selic_rates.style.format({'data': lambda t: t.strftime(std_date_format)})
selic_rates = selic_rates.loc[(selic_rates['data'].between(START_DATE_FINAL, END_DATE_FINAL))]
selic_rates['valor'] = pd.to_numeric(selic_rates['valor'])
selic_rates['valor_ajustado'] = selic_rates['valor']/100

In [193]:
selic_rates

,data,valor,valor_ajustado
1,2021-01-04,0.007469,0.000075
2,2021-01-05,0.007469,0.000075
3,2021-01-06,0.007469,0.000075
4,2021-01-07,0.007469,0.000075
5,2021-01-08,0.007469,0.000075
...,...,...,...
359,2022-06-07,0.047279,0.000473
360,2022-06-08,0.047279,0.000473
361,2022-06-09,0.047279,0.000473
362,2022-06-10,0.047279,0.000473


In [194]:
def filter_by_freq(df, frequency):
    filtered_df = df.copy()
    
    if frequency.upper() == 'DAY':
        pass
        
    else:
        date_obj = filtered_df['data'].values[0]
        day = pd.to_datetime(date_obj).day
        month = pd.to_datetime(date_obj).month
        
        if frequency.upper() == 'MONTH':
            filtered_df = filtered_df.loc[filtered_df['data'].dt.day.isin(range(day - 1, day + 1))]
        
        elif frequency.upper() == 'YEAR':
            filtered_df = filtered_df.loc[filtered_df['data'].dt.day.isin(range(day - 1, day + 1))]
            filtered_df = filtered_df.loc[filtered_df['data'].dt.month.eq(month)]
    
    return filtered_df


def calculate_interest(df, frequency, capital):
    calculations_df = filter_by_freq(df, frequency)
    answer_df = pd.DataFrame(
        [{'Date': df['data'].values[0], 'Capital': capital, 'Amount Earned': 0}]
    )
    
    if frequency.upper() == 'DAY':
        current_capital = capital
        for index, row in calculations_df.iterrows():
            current_capital *= (1 + row['valor_ajustado'])
            new_row = {
                'Date': row['data'],
                'Capital': current_capital,
                'Amount Earned': current_capital - capital
            }
            
            answer_df = answer_df.append(new_row, ignore_index=True)

    elif frequency.upper() == 'MONTH':
        pass
    
    elif frequency.upper() == 'YEAR':
        pass

    return answer_df

In [195]:
df = calculate_interest(selic_rates, PERIOD_FINAL, CAPITAL_FINAL)
df

,Date,Capital,Amount Earned
0,2021-01-04,100.000000,0.000000
1,2021-01-04,100.007469,0.007469
2,2021-01-05,100.014939,0.014939
3,2021-01-06,100.022409,0.022409
4,2021-01-07,100.029879,0.029879
...,...,...,...
359,2022-06-07,109.231179,9.231179
360,2022-06-08,109.282823,9.282823
361,2022-06-09,109.334491,9.334491
362,2022-06-10,109.386183,9.386183


In [196]:
if save_csv:
    df.to_csv(f'{filename}.csv', encoding='utf-8')

if save_xlsx:
    df.to_excel(f'{filename}.xlsx', encoding='utf-8')